In [1]:
import sys,time

string = """ S how
 C inemas
 I n
 M y
 P lace
 L isting
 E verything"""

typing_speed = 390

def slow_type(t):
    for l in t:
        sys.stdout.write(l)
        sys.stdout.flush()
        time.sleep(10.0/typing_speed)
    print('\n')

print('')
slow_type(string)


 S how
 C inemas
 I n
 M y
 P lace
 L isting
 E verything



In [1]:
import requests
import sys
import emoji
from datetime import datetime as dt
from datetime import timedelta
        
def cinemoji(cinema, place):
    d_cinema = get_dict(cinema)
    
    name = d_cinema['name']
    emoticon = emoji.emojize(d_cinema['emoji'])
    
    if place == 'before':
        return emoticon +' '+ name
    elif place == 'after':
        return name +' '+ emoticon
    elif place == 'both':
        return emoticon +' '+ name +' '+ emoticon

In [3]:
print(emoji.emojize(":popcorn:"), 'WOKI', emoji.emojize(":popcorn:"))

print(emoji.emojize(":bread:"), 'Brotfabrik', emoji.emojize(":bread:"))

print(emoji.emojize(":T-Rex:"), 'Rex', emoji.emojize(":T-Rex:"))

print(emoji.emojize(":NEW_button:"), 'Neue Filmbühne', emoji.emojize(":NEW_button:"))

🍿 WOKI 🍿
🍞 Brotfabrik 🍞
🦖 Rex 🦖
🆕 Neue Filmbühne 🆕


In [ ]:
import sys, time, random
import emoji
from datetime import datetime as dt
from datetime import timedelta
from shutil import get_terminal_size
import scimple_functions as SF

def slow_type(t):
    typing_speed = 375
    for l in t:
        sys.stdout.write(l)
        sys.stdout.flush()
        time.sleep(10.0/typing_speed)
    print('\n')

def opening():
    string = """     S how
     C inemas
     I n
     M y
     P lace
     L isting
     E verything"""

    print('')
    slow_type(string)
    
opening()

def inputexit(var):
    var = input(var)
    if var == 'exit':
        print('exited session.')
        sys.exit()
    else:
        return var


print(emoji.emojize(":T-Rex:"), 'Rex', emoji.emojize(":T-Rex:"))
print(emoji.emojize(":popcorn:"), 'WOKI', emoji.emojize(":popcorn:"))
print(emoji.emojize(":bread:"), 'Brotfabrik', emoji.emojize(":bread:"))
print(emoji.emojize(":NEW_button:"), 'Neue Filmbühne', emoji.emojize(":NEW_button:"))
print('')

imported = []

print('default is all, sort with arg [t]ime, [a]lphabet, [c]inema, [r]oom, [s]eat\n')

def routine():
    sorter_key = None
    
    while True:
        is_date = False

        p_date = inputexit('date: ').lower().replace(' ', '')

        if '--' in p_date:
            p_date, sorter_key = p_date.split('--', 1)


        if str(p_date).lower() in [l.lower() for l in SF.l_today]:
            p_date = dt.today()

        elif str(p_date).lower() in [l.lower() for l in SF.l_tomorrow]:
            p_date = dt.today() + timedelta(days=1)

        elif str(p_date).lower() in [l.lower() for l in SF.l_dayname]:
            day_hit = SF.l_dayname.index(p_date.capitalize()) %7
            today = dt.today().strftime('')

            if day_hit < dt.today().weekday():
                day_skip = 7 - dt.today().weekday() + day_hit
            elif day_hit == dt.today().weekday():
                day_skip = 7
            else:
                day_skip = day_hit - dt.today().weekday()

            p_date = dt.today() + timedelta(days=day_skip)


        if p_date == '':
            p_date = 'all'
            is_date = True

        else:
            p_date, is_date = SF.detect_date(p_date)

        if is_date:
            break
        else:
            print('invalid date')


    if p_date == 'all':
        p_timestamp = False

    else:
        print('')
        while True:
            p_time = inputexit('time: ').lower().replace(' ', '')

            if p_time == '':
                p_time = '0000'

            if ':' in p_time:
                hour, minute = p_time.split(':', 1)
            else:
                hour = p_time[:-2]
                minute = p_time[-2:]

            try:
                p_timestamp = p_date.replace(hour=int(hour), minute=int(minute), second=0, microsecond=0)
                break

            except ValueError:
                print('invalid time')            

    print('')
    while True:
        p_cinema = inputexit('cinema: ').lower().replace(' ', '').split(',')

        if p_cinema == ['']:
            p_cinema = []
            for i in SF.cinema_alias:
                p_cinema.append(i[0])

        else:
            p_cinema = list(filter(None, p_cinema))

        p_cinema = list(set([SF.is_cinema_alias(p) for p in p_cinema]))

        if False in p_cinema:
            print('invalid cinema key.')
            continue

        else:
            for p in p_cinema:
                if not SF.cinema_in_dlist(p, imported):
                    desc = 'importing ' + p + '...'
                    end = p + ' done.'
                    loader = SF.Loader(desc, end).start()

                    imported.append(SF.import_cinema_errorless(p))

                    loader.stop()
            break    

    p_title = inputexit('\ntitle: ').lower().rstrip()
    if p_title == '':
        p_title = False
        
    print('')
    
    selected_cinema = [event for cinema in imported for selected in p_cinema if selected in cinema['alias'] for event in cinema['event']]
        
    SF.show(selected_cinema, sorter_key, p_timestamp, p_title)
    

while True:
    routine()
    print('\n\n\n\n')


In [ ]:
import pyshorteners
type_tiny = pyshorteners.Shortener()
short_url = type_tiny.tinyurl.short(event_d['ticket'])
print(return_color(short_url, '39'))

In [ ]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt


timestamp = []
title = []
room = []
spec = []
location = []
ticket = []

url = 'https://www.imdb.com/title/tt11198810/'
page = urlopen(url)
html = page.read().decode("utf-8")

soup = BeautifulSoup(html, "html.parser")
print(soup)

#match_results = 
#jstring = json.loads(match_results)
#print(json.dumps(jstring, indent=3))
#pattern = r'<div class="relative">.*?</div>'
#match_results = re.findall(pattern, html, re.DOTALL)

In [20]:
woki = SF.import_cinema_errorless('woki')

In [31]:
import pyshorteners
import threading
import concurrent.futures

In [ ]:
for i in woki['event']:
    shortener(i['ticket'])

In [47]:
def shortener(url):
    type_tiny = pyshorteners.Shortener()
    short_url = type_tiny.tinyurl.short(url)
    return short_url

In [ ]:
#multithreading
threads = list()

print(len(woki['event']))

for event in woki['event']:
    x = threading.Thread(target=shortener, args=(event['ticket'],))
    threads.append(x)
    x.start()

for url, thread in enumerate(threads):
    print(url, thread)
    thread.join()

In [ ]:
for event in woki['event']:
    print(event['ticket'])

In [28]:
import rottentomatoes as rt

In [ ]:
for event in woki['event']:
    while True:
        try:
            movies = rt.tomatometer(event['title'])
            print(movies)

        except Exception as e:
            print("\r" , end="", flush=True)
            
            print('retrying..')
            time.sleep(4)

In [63]:
movie = ia.search_movie('harry potter')[0]
print(movie)
movie.infoset2keys

Harry Potter and the Sorcerer's Stone


{}

In [12]:
from imdb import Cinemagoer

In [14]:
ia = Cinemagoer()

In [ ]:
for event in woki['event']:
    while True:
        try:
            movies = ia.search_movie(event['title'])
            print(movies)

        except Exception as e:
            print("\r" , end="", flush=True)
            
            print('retrying..')
            time.sleep(4)

In [19]:
# create an instance of the Cinemagoer class
for director in the_matrix['directors']:
    print(director['name'])
print(sorted(the_matrix.keys()))
print(ia.get_movie_infoset())
ia.update(the_matrix, ['technical'])
print(the_matrix.infoset2keys['technical'])
print(the_matrix.get('tech'))

Lana Wachowski
Lilly Wachowski
['akas', 'animation department', 'art department', 'art direction', 'aspect ratio', 'assistant director', 'box office', 'camera and electrical department', 'canonical title', 'cast', 'casting department', 'casting director', 'certificates', 'cinematographer', 'color info', 'composer', 'costume department', 'costume designer', 'countries', 'country codes', 'cover url', 'director', 'distributors', 'editor', 'editorial department', 'full-size cover url', 'genres', 'imdbID', 'kind', 'language codes', 'languages', 'localized title', 'location management', 'long imdb canonical title', 'long imdb title', 'make up', 'miscellaneous crew', 'music department', 'original air date', 'original title', 'other companies', 'plot', 'plot outline', 'producer', 'production companies', 'production design', 'production manager', 'rating', 'runtimes', 'script department', 'set decoration', 'smart canonical title', 'smart long imdb canonical title', 'sound crew', 'sound mix', 's

In [14]:
import time
ia = Cinemagoer()

for i in movietitle[6:10]:
    print('##', i)

    before = time.time()
    movie = ia.search_movie(i)[0].movieID
    print(movie)
    print(time.time()-before)
    before = time.time()
    movies = ia.get_movie(movie, info=['rating'])
    #movie = ia.get_movie(movie, info=['cast'])
    #print(movie.infoset2keys)
    #print(ia.get_movie(movie, info=['rating']))
    print(movies.get('rating'))
    movies = ia.get_movie(movie)
    print(movies.infoset2keys)
    print(time.time()-before)
    before = time.time()


## the nun 2


2023-10-25 01:48:24,913 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


10160976
1.3201336860656738
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'akas', 'production companies', 'distributors', 'special effects companies', 'other companies'], 'plot': ['plot', 'synopsi

2023-10-25 01:48:28,576 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


28627002
0.8141398429870605
None
{'main': ['localized title', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'certificates', 'rating', 'votes', 'cover url', 'imdbID', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'production companies'], 'plot': ['plot']}
2.3289988040924072
## barbie


2023-10-25 01:48:31,888 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


1517268
0.9829950332641602
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'animation department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'thanks', 'akas', 'production companies', 'distributors', 'special effects companies', 'other companie

2023-10-25 01:48:36,050 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


15398776
1.0211632251739502
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'animation department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'thanks', 'akas', 'top 250 rank', 'production companies', 'distributors', 'special ef

In [ ]:
import time
ia = Cinemagoer()
print(ia.get_movie_infoset())

for i in movietitle:
    print('##', i)

    before = time.time()
    movie = ia.search_movie(i)[0]
    print(movie)
    print('found')
    print(time.time()-before)
    before = time.time()

    ia.update(movie, info=['rating'])
    print(time.time()-before)
    before = time.time()

    print('info', movie.current_info)
    print(time.time()-before)
    before = time.time()

    print(type(movie))

    print(time.time()-before)
    before = time.time()

    try:
        print(movie.infoset2keys)
    except KeyError:
        print('this')
    print('\n')

In [44]:
try:
    print(movie.infoset2keys)
except KeyError:
    print('this')
print('\n')

{}




In [35]:
brot = bf.brotfabrik()

ia = Cinemagoer()
print(ia.get_movie_infoset())

print(resp)

for i in movietitle:
    try:
        print('##', i)
        
        movie = ia.search_movie(i)[0]
        for key, value in movie.items():
            print(key, value)
        print('\n')
        
        rtmovie = rt.Movie(i)
        if movie['year'] == rt.year_released(i):
            print(rtmovie, '\n')
        else:
            raise KeyError
        
    except KeyError:
        try:
            ia.update(movie, info=['main'][0])
            print(movie.get('original title'))
            movie = rt.Movie(movie.get('original title'))
            print(movie, '\n')
        except:
            print('no ratings on tomatoes')
            continue
        continue

['airing', 'akas', 'alternate versions', 'awards', 'connections', 'crazy credits', 'critic reviews', 'episodes', 'external reviews', 'external sites', 'faqs', 'full credits', 'goofs', 'keywords', 'list', 'locations', 'main', 'misc sites', 'news', 'official sites', 'parents guide', 'photo sites', 'plot', 'quotes', 'recommendations', 'release dates', 'release info', 'reviews', 'sound clips', 'soundtrack', 'synopsis', 'taglines', 'technical', 'trivia', 'tv schedule', 'video clips', 'vote details']


NameError: name 'resp' is not defined

In [171]:
brot = bf.brotfabrik()

ia = Cinemagoer()
print(ia.get_movie_infoset())

for i in brot['title']:
    print(i)
    movie = ia.search_movie(i)
    for m in movie:
        print(m['title'], type(m))

        for key, value in m.items() :
            print (key, value)

        print('\n')

NameError: name 'bf' is not defined

In [ ]:
ia = Cinemagoer()
print(ia.get_movie_infoset())

movie = ia.search_movie('rose')

print(movie, type(movie[2]))
for m in movie:
    print(m['title'], type(m))
    
    for key, value in m.items() :
        print (key, value)
        
    print('\n')

In [5]:
# create an instance of the Cinemagoer class
ia = Cinemagoer()

# get a movie
movie = ia.get_movie('0133093')

# print the names of the directors of the movie
print('Directors:')
for director in movie['directors']:
    print(director['name'])

# print the genres of the movie
print('Genres:')
for genre in movie['genres']:
    print(genre)

# search for a person name
people = ia.search_person('Mel Gibson')
for person in people:
    print(person.personID, person['name'])

Directors:
Lana Wachowski
Lilly Wachowski
Genres:
Action
Sci-Fi
0000154 Mel Gibson
12550288 Mel Gibson
0317112 Mel Gibson
2153167 Mel Gibson
15271356 Mel Gibson
4798507 Mel A. Gibson
3323885 Adam Taylor
2800055 Mel Gibson
3742421 Melina Mari Bryant
0317096 Meg Gibson
0879085 Tyrese Gibson
0684545 Julie Pinson
8908771 Millie Gibson
0316995 Hoot Gibson
0316945 Donal Gibson
4047961 Noel Gibson
6521923 Mel Gibbon
2988370 Mal Gibson
0316932 Dean Paul Gibson
0316878 Cal Gibson


In [36]:
for i in movietitle:
    print('-'*48)
    movieinput = i

    resp = requests.get('https://rotten-tomatoes-api.ue.r.appspot.com/search/' + movieinput)

    print(resp.json(), '\n')
    print(len(resp.json().get('movies')))

    i = 0
    if len(resp.json().get('movies')) > 1:
        for m in resp.json().get('movies'):
            i += 1
            print(f'{i:2.0f}.', m.get('name'))
            print(' '*3, 'release:', m.get('year'))

        hit = int(abort(input('-> select wich file: ')))
        print(resp.json().get('movies')[hit - 1])

    else:
        print(resp.json().get('movies')[0].get('name'))


------------------------------------------------
{'movies': [{'name': 'Killers of the Flower Moon', 'tomatometer': 92, 'audience_score': 85, 'weighted_score': 89, 'genres': ['Crime', 'Drama'], 'rating': 'R', 'duration': '3h 26m', 'year': '2023', 'actors': ['Leonardo DiCaprio', 'Robert De Niro', 'Jesse Plemons', 'Lily Gladstone', 'Tantoo Cardinal'], 'directors': ['Martin Scorsese']}]} 

1
Killers of the Flower Moon
------------------------------------------------
{'movies': []} 

0


IndexError: list index out of range

In [1]:
import time
def import_url(url):
    while True:
        try:
            page = urlopen(url)

        except Exception as e:
            print("\r" , end="", flush=True)
            
            print(e)
            time.sleep(4)
        else:
            print('finally.')
            return page
        

In [8]:
url =         
import_url(url)

In [ ]:
soup = BeautifulSoup(html, 'html5lib')
print(soup.prettify())

In [ ]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt
from datetime import timedelta
from bs4 import BeautifulSoup

url = "https://www.kinopolis.de/bn/programm"
html = urlopen(url).read().decode("utf-8")

pattern = r'<section class="bg.*?<section class="bg'
match_results = re.findall(pattern, html, re.DOTALL)

In [ ]:
def ():
    result = {
        "alias": [],
        "event"
    }

In [ ]:
<section class="bg-2  movie mb-2"  data-fsk="12" data-genre='["komödie"]' data-version='["2d"]'>

In [202]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt
from datetime import timedelta
from bs4 import BeautifulSoup

def None_AttributeError_cnt(function):
    try:
        return function.contents[0]
    except:
        return None

def kinopolis():
    result = {
    "alias": ['kinopolis', 'kp'],
    "event": []
    }

    url = "https://www.kinopolis.de/bn/programm"
    html = import_url(url).read().decode("utf-8")
    
    soup = BeautifulSoup(html, "html.parser")
    match_results = soup.find_all("section", {"class": "bg-2 movie mb-2"})
    print(match_results)
    
    for program in match_results:
        title = program.find("a", {"class": "hl-link", "href": True}).contents[0]

        subtitle = program.find("span", {"class": "hl--sub"})
        if subtitle and len(subtitle.contents) > 0:
            subtitle = subtitle.contents[0]
        else:
            subtitle = None

        jspec = program.find_all("div", {"class": "movie__specs-el"})
        all_spec = []
        for i in jspec:
            if 'FSK' in str(i):
                all_spec.append(str(i).split('</span> ', 1)[1].split('<', 1)[0])

            elif 'Dauer' in str(i):
                runtime = str(i).split('</span> ', 1)[1].split('<', 1)[0].replace('Minuten', '').replace(' ', '')

        spec_caption = []
        caption_wrap = program.find_all("div", {"class": "caption__wrap"})
        if caption_wrap:
            for caption in caption_wrap:
                tmp = caption.find("b")
                if tmp:
                    caption_tag = tmp.contents[0]
                    caption_text = caption.find("div", {"class": "caption__text"}).contents[0].replace(' - Atmos', '')

                    for t in [title, subtitle]:
                        caption_text = "".join(caption_text.split(t)).lstrip().rstrip()

                    if caption_tag in caption_text:
                        caption_text = caption_text.split(caption_tag)[1].replace(':', '')

                    if [caption_tag, caption_text] not in spec_caption:
                        spec_caption.append([caption_tag, caption_text])

                else:
                    string = [c.string for c in caption if c.string not in [None, '\n']]
                    if len(string) == 1:
                        string = string[0]
                    else:
                        string = ': '.join([c.string for c in caption if c != '\n'])
                    spec_caption.append(['tag_all', string])


        l_day = []
        l_price = []

        table = program.find_all("div", {"class": "slider slider-6 prog-nav"})
        pattern = r'data-performance-ids=".*?</div></div></div>'
        for slider in re.findall(pattern, str(table)):        
            day = re.search('<div class="prog-nav__day">(.*?)</div>', slider).group(1)

            if 'Heute' in day:
                day = dt.today()

            elif 'Morgen' in day:
                day = dt.today() + timedelta(+1)

            else:
                day = dt.strptime(day.split(' ')[-1], '%d.%m.')

            l_day.append(day.replace(year=dt.today().year, second=0, microsecond=0))


        i = -1
        for day_slide in program.find_all("div", {"class": "prog-day__wrapper"}):
            i += 1
            day = l_day[i]

            for screen in day_slide.find_all("div", {"class": "prog2__cont", "data-performance-id": True}):            
                time = None_AttributeError_cnt(screen.find("div", {"class": "prog2__time"}))
                if time:
                    hour, minute = time.split(':', 1)
                    timestamp = day.replace(hour=int(hour), minute=int(minute))

                else:
                    timestamp = None

                try:
                    room = re.search('<div>(.*?)</div>', str(screen)).group(1)
                except:
                    room = None

                seat = None_AttributeError_cnt(screen.find("div", {"class": "prog2__seats"}))

                load = None_AttributeError_cnt(screen.find("div", {"class": "prog2__scale scale-100"}))

                try:
                    tmp = screen.find("a", {"data-version": True, "href": True})['data-version'].replace('[', '').replace(']', '')
                    tmp = tmp.replace('"', '').upper().split(',')

                    if '2D' in tmp:
                        tmp.remove('2D')

                    if all(i in tmp for i in ['OV', 'OMU']):
                        tmp.remove('OV')

                    spec = []                
                    [spec.append(i) for i in tmp if i not in all_spec + spec]

                    for caption in spec_caption:
                        if caption[0] == 'tag_all':
                            all_spec.append(caption[1])

                        elif any(caption[0].lower() == t.lower() for t in spec):
                            spec = [caption[0] +': '+ caption[1] if s.lower() == caption[0].lower() else s for s in spec]

                    spec = ', '.join(all_spec + spec)
                except:
                    spec = ', '.join(all_spec)

                try:
                    ticket = 'https://www.kinopolis.de' + screen.find("a", {"data-version": True, "href": True})['href']
                except:
                    ticket = None

                price = None_AttributeError_cnt(screen.find("b"))
                if price:
                    price = price + '€'


                page = {
                    "name": 'KINOPOLIS',
                    "short name": 'kinopolis',
                    "emoji": ':burrito:',
                    "timestamp": timestamp,
                    "title": title,
                    "subtitle": subtitle,
                    "room": room,
                    "seat": seat,
                    "runtime": runtime,
                    "spec": spec,
                    "location": 'KINOPOLIS',
                    "ticket": ticket,
                    "load": load,
                    "price": price
                }

                result['event'].append(page)
                
    return result

In [220]:
dt.today().hour

0

In [51]:
import time
while True:
    l_day = []
    url = "https://www.kinopolis.de/bn/programm"
    html = import_url(url).read().decode("utf-8")
    print(dt.today())
    
    pattern = r'<section class="bg.*?<section class="bg'
    match_results = re.findall(pattern, html, re.DOTALL)

    for movie in match_results:
        program = BeautifulSoup(movie, "html.parser")   
        
        table = program.find_all("div", {"class": "slider slider-6 prog-nav"})
        pattern = r'data-performance-ids=".*?</div></div></div>'
        for slider in re.findall(pattern, str(table)):
            day = re.search('<div class="prog-nav__day">(.*?)</div>', slider).group(1)

            if 'Heute' in day:
                day = dt.today()

            elif 'Morgen' in day:
                day = dt.today() + timedelta(+1)

            else:
                day = dt.strptime(day.split(' ')[-1], '%d.%m.')

            l_day.append(day.replace(year=dt.today().year, second=0, microsecond=0))
    print(l_day)
    print(l_day[0].day )
    if l_day[0].day == '14':
        sys.exit()
    time.sleep(60)

<urlopen error [Errno 54] Connection reset by peer>
finally.
2024-03-14 00:59:12.101287
[datetime.datetime(2024, 3, 15, 0, 59), datetime.datetime(2024, 3, 15, 0, 0), datetime.datetime(2024, 3, 16, 0, 0), datetime.datetime(2024, 3, 17, 0, 0), datetime.datetime(2024, 3, 18, 0, 0), datetime.datetime(2024, 3, 19, 0, 0), datetime.datetime(2024, 3, 15, 0, 59), datetime.datetime(2024, 3, 15, 0, 0), datetime.datetime(2024, 3, 16, 0, 0), datetime.datetime(2024, 3, 17, 0, 0), datetime.datetime(2024, 3, 18, 0, 0), datetime.datetime(2024, 3, 19, 0, 0), datetime.datetime(2024, 3, 15, 0, 59), datetime.datetime(2024, 3, 15, 0, 0), datetime.datetime(2024, 3, 16, 0, 0), datetime.datetime(2024, 3, 17, 0, 0), datetime.datetime(2024, 3, 18, 0, 0), datetime.datetime(2024, 3, 19, 0, 0), datetime.datetime(2024, 3, 15, 0, 59), datetime.datetime(2024, 3, 15, 0, 0), datetime.datetime(2024, 3, 16, 0, 0), datetime.datetime(2024, 3, 17, 0, 0), datetime.datetime(2024, 3, 18, 0, 0), datetime.datetime(2024, 3, 19, 0,

KeyboardInterrupt: 